# *Aufgabe: Probabilistische Methoden*

0. Imports

In [20]:
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support
import matplotlib.pyplot as plt
import numpy as np
from sklearn import metrics
from scipy import stats

### Aufgabe 1. Gaussian Naive Bayes

1. Import and Train/Test split

In [21]:
census = pd.read_csv("../../../data/census.csv")[['age', 'hours-per-week', 'target']]
census["target"] = census["target"].map({" <=50K":0," >50K":1})
census.head(5)

,age,hours-per-week,target
0,39,40,0
1,50,13,0
2,38,40,0
3,53,40,0
4,28,40,0


In [22]:
x_train, x_test, y_train, y_test = train_test_split(census.drop(["target"],axis=1),census["target"])
print('X Train: {}'.format(x_train.shape)) 
print('Y Train: {}'.format(y_train.shape)) 
print('X Test: {}'.format(x_test.shape)) 
print('Y Test: {}'.format(y_test.shape))

# Normalize: Age und hours-per-week
minmax = MinMaxScaler()

x_train['age'] = minmax.fit_transform(x_train['age'].to_numpy().reshape(-1,1))
x_train['hours-per-week'] = minmax.fit_transform(x_train['hours-per-week'].to_numpy().reshape(-1,1))

x_test['age'] = minmax.fit_transform(x_test['age'].to_numpy().reshape(-1,1))
x_test['hours-per-week'] = minmax.fit_transform(x_test['hours-per-week'].to_numpy().reshape(-1,1))

X Train: (24420, 2)
Y Train: (24420,)
X Test: (8141, 2)
Y Test: (8141,)


2. Gaussian Naive Bayes:

In [30]:
def calculate_probability(prior_high, prior_low, x):
    proba_high = np.log(prior_high*age_density_high(x[0])*hours_density_high(x[1]))
    proba_low = np.log(prior_low*age_density_low(x[0])*hours_density_low(x[1]))
    if proba_high >= proba_low:
        return 1
    else: 
        return 0
    
# Prior Probabilities:

data_shape = y_train.shape[0]
prior_probability_high_income = y_train.loc[y_train == 1].shape[0]/data_shape
prior_probability_low_income = y_train.loc[y_train == 0].shape[0]/data_shape
print("Prior probability high income: {:.2f}".format(prior_probability_high_income*100))
print("Prior probability low income: {:.2f}".format(prior_probability_low_income*100))

data_train = x_train
data_train['target'] = y_train
age_density_high = stats.kde.gaussian_kde(data_train.loc[data_train["target"]==1]["age"])
age_density_low = stats.kde.gaussian_kde(data_train.loc[data_train["target"]==0]["age"])
hours_density_high = stats.kde.gaussian_kde(data_train.loc[data_train["target"]==1]["hours-per-week"])
hours_density_low = stats.kde.gaussian_kde(data_train.loc[data_train["target"]==0]["hours-per-week"])

# Prediction:
predicted = []
for element in x_test.to_numpy():
    predicted.append(calculate_probability(prior_probability_high_income, prior_probability_low_income, element))
     
print("Accuracy score: {}".format(accuracy_score(predicted, y_test)))

# Confusion matrix

cm = confusion_matrix(predicted, y_test)
tn, fp, fn, tp = cm.ravel()

print("Confusion matrix: \n{}".format(cm))    
print("Precision: {}".format(tp/(tp + fp)))
print("Recall: {}".format(tp/(tp + fn)))    

Prior probability high income: 24.20
Prior probability low income: 75.80
Accuracy score: 0.7690701388035868
Confusion matrix: 
[[5731 1402]
 [ 478  530]]
Precision: 0.2743271221532091
Recall: 0.5257936507936508
